In [1]:
import numpy as np
import librosa
import tensorflow as tf
import soundfile as sf

# Load the trained model with proper loss definition
model_path = 'ddae_model.h5'  # Replace with the correct path to your model
ddae = tf.keras.models.load_model(model_path, custom_objects={
    'mse': tf.keras.losses.MeanSquaredError()
})
print(f"Model loaded from {model_path}")

# Helper functions
def load_audio(file_path, sr=16000):
    """Load audio file and return the signal."""
    audio, _ = librosa.load(file_path, sr=sr)
    return audio

def compute_spectrogram(audio, n_fft=512, hop_length=256):
    """Compute magnitude and phase of the spectrogram."""
    spectrogram = librosa.stft(audio, n_fft=n_fft, hop_length=hop_length)
    magnitude = np.abs(spectrogram)
    phase = np.angle(spectrogram)
    return magnitude, phase

def preprocess_data(audio, n_fft=512, hop_length=256):
    """Preprocess audio by computing its spectrogram."""
    magnitude, phase = compute_spectrogram(audio, n_fft, hop_length)
    return magnitude, phase

def denoise_audio(ddae, noisy_audio, n_fft=512, hop_length=256):
    """Denoise noisy audio using the trained model."""
    # Compute the spectrogram of the noisy audio
    noisy_magnitude, noisy_phase = preprocess_data(noisy_audio, n_fft, hop_length)
    noisy_magnitude = noisy_magnitude.T[..., np.newaxis]  # Reshape for model input
    
    # Predict the clean magnitude spectrogram
    denoised_magnitude = ddae.predict(noisy_magnitude)
    denoised_magnitude = denoised_magnitude.squeeze().T  # Reshape back to original dimensions
    
    # Reconstruct the audio using the denoised magnitude and original phase
    denoised_spectrogram = denoised_magnitude * np.exp(1j * noisy_phase)
    denoised_audio = librosa.istft(denoised_spectrogram, hop_length=hop_length)
    return denoised_audio

# Specify the noisy audio file path
noisy_audio_path = 'p232_011.wav'  # Replace with the path to your noisy audio file
output_denoised_path = 'new_denoised_audio.wav'

# Load the noisy audio
noisy_audio = load_audio(noisy_audio_path)

# Perform denoising
denoised_audio = denoise_audio(ddae, noisy_audio)

# Save the denoised audio
sf.write(output_denoised_path, denoised_audio, 16000)
print(f"Denoised audio saved to {output_denoised_path}")

Model loaded from ddae_model.h5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step 
Denoised audio saved to new_denoised_audio.wav
